In [1]:
import numpy as np
import pandas as pd
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Embedding, LSTM, Bidirectional, Dense
from keras.models import Sequential, load_model
from keras_contrib.layers import CRF
from keras.callbacks import ModelCheckpoint
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

Using TensorFlow backend.
C:\Users\MANHLK\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\MANHLK\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\MANHLK\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\MANHLK\Anaconda3\lib\site-packages\tensorflow\python\framework\dtype

In [2]:
# Config
max_len = 80
batch_size = 64
epochs = 100
embedding = 64
hidden_size = 50

## Load data

In [3]:
def load_data(path_file = r'D:\Documents\Đồ án tốt nghiệp\Mobile-pricing\Mobile\NER\tagger-model\train.txt'):
    sentences = []
    sentence = []
    word = []
    tag = []
    with open(path_file, encoding='utf8') as file_in:
        for line in file_in:
            if line == '\n': 
                sentences.append(sentence)
                sentence = []
            else:
                w, t = line.lower().split()
                word.append(w)
                tag.append(t)
                sentence.append((w, t))
    return sentences, word, tag

## Process data

In [8]:
def process_data(sentence, word, tag):
    # Tạo dict word, tag
    word = list(set(word))
    tag = list(set(tag))
    num_word = len(word) + 2
    num_tag = len(tag) + 1
    
    # Tạo dict của word to index
    word_index = {w: i + 2 for i, w in enumerate(word)}
    word_index['UNK'] = 1
    word_index['PAD'] = 0
    # index to word
    index_word = {i: w for w, i in word_index.items()}
    # word to vector
    X = [[word_index[w[0]] for w in s]for s in sentence]
    # padding 
    X = pad_sequences(X, maxlen=max_len, padding='post', truncating='post', value = word_index['PAD'])
    
    # Tạo dict của tag to index 
    tag_index = {t : i + 1 for i, t in enumerate(tag)}
    tag_index['PAD'] = 0
    # Tạo index to tag
    index_tag = {i : t for t, i in tag_index.items()}
    # tag to vector
    y = [[tag_index[t[1]] for t in s]for s in sentence]
    # padd
    y = pad_sequences(y, maxlen=max_len, padding='post', truncating='post', value=tag_index['PAD'])
    
    # Chuyển y về dạng one hot vector
    y = [to_categorical(i, num_classes=num_tag) for i in y]
    return X, y, word_index, index_word, tag_index, index_tag, num_tag, num_word

In [9]:
def process_data_test(sentence, word_index, tag_index, word):
    # word to vector
    X = []
    for s in sentence:
        line = []
        for w in s:
            if(w[0] not in set(word)):
                line.append(1)
            else:
                line.append(word_index[w[0]])
        X.append(line)
                
    # padding 
    X = pad_sequences(X, maxlen=max_len, padding='post', truncating='post', value = word_index['PAD'])
    
    # tag to vector
    y = [[tag_index[t[1]] for t in s]for s in sentence]
    # padd
    y = pad_sequences(y, maxlen=max_len, padding='post', truncating='post', value=tag_index['PAD'])
    
    # Chuyển y về dạng one hot vector
    y = [to_categorical(i, num_classes=num_tag) for i in y]
    return X, y

## Build model

In [10]:
def build_model(num_word, num_tag):
    model = Sequential()
    model.add(Embedding(input_dim = num_word+1, output_dim=embedding, input_length=max_len))
    model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
    model.add(Dense(hidden_size, activation='relu'))
    crf = CRF(num_tag)
    model.add(crf)
    model.summary()
    model.compile('adam', loss=crf.loss_function, metrics=[crf.accuracy])
    return model

In [11]:
sentence_train, word, tag = load_data()
sentence_test , word_test, tag_test = load_data(r'D:\Documents\Đồ án tốt nghiệp\Mobile-pricing\Mobile\NER\tagger-model\test.txt')
X_train, y_train, word_index, index_word, tag_index, index_tag, num_tag, num_word = process_data(sentence_train, word, tag)
X_test, y_test = process_data_test(sentence_test, word_index, tag_index, word)

In [14]:
if not os.path.exists("model.hdf5"):
    model = build_model(num_word, num_tag)
    checkpoint = ModelCheckpoint(filepath = 'model.hdf5', save_best_only = True, monitor='val_loss')
    history = model.fit(X_train, np.array(y_train), batch_size=batch_size, epochs=epochs,
                        validation_split=0.1, callbacks=[checkpoint])
else:
    model = build_model(num_word, num_tag)
    model.load_weights("model.hdf5")

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 80, 64)            197056    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 80, 100)           46000     
_________________________________________________________________
dense_1 (Dense)              (None, 80, 50)            5050      
_________________________________________________________________
crf_1 (CRF)                  (None, 80, 36)            3204      
Total params: 251,310
Trainable params: 251,310
Non-trainable params: 0
_________________________________________________________________


C:\Users\MANHLK\Anaconda3\lib\site-packages\keras_contrib\layers\crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
C:\Users\MANHLK\Anaconda3\lib\site-packages\keras_contrib\layers\crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


Instructions for updating:
Use tf.cast instead.
Train on 1440 samples, validate on 161 samples
Epoch 1/100
1440/1440 [==============================] - 8s 5ms/step - loss: 2.8981 - crf_viterbi_accuracy: 0.4625 - val_loss: 1.7494 - val_crf_viterbi_accuracy: 0.5700
Epoch 2/100
1440/1440 [==============================] - 5s 4ms/step - loss: 1.2794 - crf_viterbi_accuracy: 0.7034 - val_loss: 1.0328 - val_crf_viterbi_accuracy: 0.7763
Epoch 3/100
1440/1440 [==============================] - 5s 4ms/step - loss: 0.9406 - crf_viterbi_accuracy: 0.7805 - val_loss: 0.9066 - val_crf_viterbi_accuracy: 0.7788
Epoch 4/100
1440/1440 [==============================] - 5s 3ms/step - loss: 0.8453 - crf_viterbi_accuracy: 0.7902 - val_loss: 0.8321 - val_crf_viterbi_accuracy: 0.8020
Epoch 5/100
1440/1440 [==============================] - 5s 4ms/step - loss: 0.7687 - crf_viterbi_accuracy: 0.8088 - val_loss: 0.7554 - val_crf_viterbi_accuracy: 0.8055
Epoch 6/100
1440/1440 [==============================] - 5s 

1440/1440 [==============================] - 7s 5ms/step - loss: 0.0139 - crf_viterbi_accuracy: 0.9945 - val_loss: 0.2661 - val_crf_viterbi_accuracy: 0.9455
Epoch 96/100
1440/1440 [==============================] - 7s 5ms/step - loss: 0.0137 - crf_viterbi_accuracy: 0.9942 - val_loss: 0.2610 - val_crf_viterbi_accuracy: 0.9440
Epoch 97/100
1440/1440 [==============================] - 7s 5ms/step - loss: 0.0140 - crf_viterbi_accuracy: 0.9941 - val_loss: 0.2724 - val_crf_viterbi_accuracy: 0.9449
Epoch 98/100
1440/1440 [==============================] - 8s 5ms/step - loss: 0.0130 - crf_viterbi_accuracy: 0.9943 - val_loss: 0.2750 - val_crf_viterbi_accuracy: 0.9429
Epoch 99/100
1440/1440 [==============================] - 7s 5ms/step - loss: 0.0133 - crf_viterbi_accuracy: 0.9945 - val_loss: 0.2623 - val_crf_viterbi_accuracy: 0.9443
Epoch 100/100
1440/1440 [==============================] - 6s 4ms/step - loss: 0.0120 - crf_viterbi_accuracy: 0.9951 - val_loss: 0.2699 - val_crf_viterbi_accuracy:

## Kiểm thử

In [15]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=-1)
y_test_true = np.argmax(y_test, axis=-1)

# Kiểm thử F1-score
y_pred = [[index_tag[i] for i in row]for row in y_pred]
y_test_true = [[index_tag[i] for i in row]for row in y_test_true]
print('F1-score:', f1_score(y_test_true, y_pred))

F1-score: 0.7855375832540439


C:\Users\MANHLK\Anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: b-type seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\MANHLK\Anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: i-type seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\MANHLK\Anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: b-nhan_hieu seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\MANHLK\Anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: b-name seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\MANHLK\Anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: b-model seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\MANHLK\Anaconda3\lib\site-packages\